In [1]:
# NOTEBOOK IMPORTS
import os
import glob
import numpy as np
from tqdm.notebook import tqdm
from shutil import copyfile

# IMAGE IMPORTS
import cv2
from PIL import Image

# GIS IMPORTS
import fiona
import pyproj
from affine import Affine
from shapely.geometry import shape, mapping
from shapely.geometry import Point, LineString
from shapely.ops import transform, nearest_points, snap
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from scipy.spatial import cKDTree

# PLOTTING IMPORTS
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# CUSTOM UTILITIES
from WorldFileUtils import *
from GeometryUtils import *
from icp import *

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
templates_dir = "data/templates/"
tempfiles_dir = "tempfiles/"

boundary_shapefile = f"{templates_dir}HCAD_Harris_County_Boundary.shp"
boundary_points    = f'{tempfiles_dir}boundary_points.shp'

In [3]:
line = fiona.open(boundary_shapefile)

firstline = line.next()
first = shape(firstline['geometry']).boundary

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:3857')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
first = transform(project, first)

# length of the LineString
length = first.length

point_boundary_list = list()

if not os.path.exists(boundary_points):
    for distance in tqdm(range(0,int(length),5)):
        point = first.interpolate(distance)   
        point_boundary_list.append(point)
        point_boundary_gdf = gpd.GeoDataFrame(geometry=point_boundary_list)
        point_boundary_gdf.to_file(boundary_points)

C:\Users\franc\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
def normalize_geometry(gdf, sp):
    """
    Subtract a point from the geometry of each point in the GeoDataFrame.

    Args:
        gdf (geopandas.GeoDataFrame): GeoDataFrame containing points.
        point (tuple): Tuple of X and Y coordinates of the point.

    Returns:
        geopandas.GeoDataFrame: GeoDataFrame with updated geometries.
    """
    
    new_x = gdf.geometry.geometry.x - sp.geometry.x.to_numpy()
    new_y = gdf.geometry.geometry.y - sp.geometry.y.to_numpy()
    # Subtract the given point from each geometry
    updated_geometries = [Point(x, y) for x,y in zip(new_x, new_y)]

    # Create a new GeoDataFrame with the updated geometries
    updated_gdf = gdf.copy()
    updated_gdf.geometry = updated_geometries

    return updated_gdf

def normalize_geometry(gdf, ot):
    """
    Warp by original transform
    """
    
    ot = ~ot
    geometry = gdf.geometry.affine_transform([ot.a, ot.b, ot.d, ot.e, ot.c, ot.f])
    gdf['geometry'] = geometry
    return gdf

In [5]:
def getActualTransform(ot, ho):
    ot_a = Affine.from_gdal(*ot)
    ho = ho.flatten()
    # t = [[ho[2]] * ho[5]]# ho[2] = ho[2] * -1
    
    # translation = np.array(ot_a).flatten()# Affine.translation(ho[2], ho[5])
    # translation[2] = ho[2]
    # translation[5] = ho[5]
    # translation = Affine(*translation[:6])
    
    # ho[2] = 0
    # ho[5] = 0
    
    ho_a = Affine(*ho[:6])
    return ot_a * ho_a

In [6]:
%%timeit -n 1 -r 1

# BOUNDARY SHAPEFILE AND BUFFER
boundary_shp = gpd.read_file(boundary_shapefile).to_crs("EPSG:3857")
boundary_buf = boundary_shp.boundary.buffer(1000)

initial_filename = f"{tempfiles_dir}border.png"
current_filename = initial_filename

plot = False

for i in range(5):
    
    output_filename = f"{tempfiles_dir}/out/border_{i}.png"
    
    print("Original Transform")
    # GET ORIGINAL TRANSFORM FROM IMAGE AND CONVERT TO AFFINE OBJECT
    original_transform, _ = get_geotransform_from_tfw(current_filename[:-3]+"pgw")
    ot = Affine.from_gdal(*original_transform)
    
    print("Coords")
    # PIXEL COORDINATES FROM TILE (ONLY CANDIDATES), NORMALIZED BY INVERSE AFFINE FOR IMAGE COORDINATES
    # coords_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(coords[:, 0] - np.mean(coords[:, 0]), coords[:, 1] - np.mean(coords[:, 1])))
    coords = get_true_pixel_coordinates(current_filename, polygon=boundary_buf)    
    coords_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1] ))
    coords_gdf = normalize_geometry(coords_gdf, ot)
    coords_gdf = coords_gdf.sample(n=1000)
    
    # ALL PIXELS FROM TILE, NORMALIZED BY INVERSE AFFINE FOR IMAGE COORDINATES
    coords_full = get_true_pixel_coordinates(current_filename)
    coords_full_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(coords_full[:, 0], coords_full[:, 1]))
    coords_full_gdf = normalize_geometry(coords_full_gdf, ot)
                                  
    # PIXEL COORDINATES FROM CLOSEST POINT IN LINE
    # point_boundary_gdf = normalize_geometry(point_boundary_gdf, point_boundary_gdf.dissolve().centroid)
    point_boundary_gdf = gpd.read_file(boundary_points)
    point_boundary_gdf = normalize_geometry(point_boundary_gdf, ot)
    boundary_line = LineString(point_boundary_gdf.geometry)
    boundary_points_matching = find_points_on_linestring(boundary_line, coords_gdf.geometry)
    boundary_points_matching_gdf = gpd.GeoDataFrame(geometry=boundary_points_matching)
    
    
    if plot:
        fig, ax = plt.subplots(figsize=(30, 30))
        # boundary_buf.plot(ax=ax, label="Zone of Interest for Candidate Points")
        # boundary_shp.plot(ax=ax, color="None", edgecolor='black', label="True Boundary")
        coords_gdf.plot(ax=ax, marker='.', color='red', markersize=2, label="Candidate Points")
        boundary_points_matching_gdf.plot(ax=ax, marker='x', color="green", label="Candidate Targets")
        ax.invert_yaxis()
        ax.legend()
        plt.show()
    
    # COORDINATE ARRAY 
    boundary_points_matching_coords = np.array([np.array([m.x, m.y]) for m in boundary_points_matching])
    
    # TO AND FROM POINTS
    to_points = np.array(boundary_points_matching_coords).astype(np.float32)
    from_points = np.array([np.array([cg.x, cg.y]).astype(np.float32) for cg in coords_gdf.geometry])
    
    # CALCULATE HOMOGRAPHY
    test = cv2.findHomography(from_points, to_points, cv2.RANSAC,1000)
    
    points_to_project = np.vstack((coords_full_gdf.geometry.x.to_numpy().T, coords_full_gdf.geometry.y.to_numpy().T)).T
    homography = test[0]
    print(homography)
    
    original_transform, _ = get_geotransform_from_tfw(current_filename[:-3]+"pgw")
    
    current_transform = getActualTransform(original_transform, homography.flatten()[:6])
    
    copyfile(current_filename, output_filename)
    write_world_file_from_affine(current_transform, output_filename[:-3]+"pgw")
    current_filename = output_filename
    
    
    if plot:
        points_to_project_homogeneous = np.hstack((points_to_project, np.ones((points_to_project.shape[0], 1))))
        # Project the points using the homography matrix
        projected_points_homogeneous = np.dot(homography, points_to_project_homogeneous.T).T

        # Convert homogeneous coordinates to Cartesian coordinates
        projected_points = (projected_points_homogeneous[:, :2] / projected_points_homogeneous[:, 2:]).astype(np.int)

        projected_geometries = [Point(x, y) for x, y in projected_points]
        projected_gdf = gpd.GeoDataFrame(geometry=projected_geometries)
        fig, ax = plt.subplots(figsize=(30, 30))
        # boundary_buf.plot(ax=ax)
        # boundary_shp.plot(ax=ax, color="None", edgecolor='black')
        coords_full_gdf.plot(ax=ax, marker='.', color='red', markersize=2)
        # boundary_points_matching_gdf.plot(ax=ax, marker='x', color="green")
        projected_gdf.plot(ax=ax, marker='x', color="green", markersize=2)
        ax.invert_yaxis()
        plt.show()

Original Transform
Coords


  0%|          | 0/1000 [00:00<?, ?it/s]

[[ 1.00021077e+00  1.43671272e-02 -4.59166290e+01]
 [-1.65616996e-03  1.00703537e+00 -3.91813965e+00]
 [-4.14600337e-07  1.53765875e-06  1.00000000e+00]]
Original Transform
Coords


  0%|          | 0/1000 [00:00<?, ?it/s]

[[ 9.99680273e-01  1.45936693e-02 -4.42547901e+01]
 [-2.06181087e-03  1.00705654e+00 -4.33745691e-01]
 [-4.93936303e-07  1.65419324e-06  1.00000000e+00]]
Original Transform
Coords


...

[[ 9.92946063e-01  1.36768969e-02 -9.73161366e+00]
 [-2.50134486e-04  1.00316052e+00  1.39495036e+00]
 [-7.60150980e-07  1.56521804e-06  1.00000000e+00]]
Original Transform
Coords


  0%|          | 0/1000 [00:00<?, ?it/s]

[[ 9.98324422e-01  1.06694240e-02 -1.79002486e+01]
 [-1.09324524e-03  1.00531510e+00  2.14402752e+00]
 [-5.20620163e-07  1.50300898e-06  1.00000000e+00]]
Original Transform
Coords


  0%|          | 0/1000 [00:00<?, ?it/s]

[[ 9.88700390e-01  8.60170807e-03  1.64707012e+01]
 [-1.72182912e-03  9.97384996e-01  1.77359795e+01]
 [-9.76015583e-07  9.90436812e-07  1.00000000e+00]]
2min 28s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
to_points

NameError: name 'to_points' is not defined

In [ ]:
from_points

In [ ]:
homography = test[0]
current_transform = Affine.from_gdal(*homography.flatten()[:6])

print(homography)
print(current_transform)